In [1]:
import numpy as np
import pandas as pd
import ta

In [2]:
df = pd.read_csv('data/BTCUSDT_data.csv')
df.drop(["Unnamed: 0", "index"], axis = 1, inplace = True)

df

,Name,Time,Open,High,Low,Close,Volume
0,BTCUSDT,2022-12-06 04:51:00,17004.99,17009.01,17002.34,17007.29,93.81524
1,BTCUSDT,2022-12-06 04:52:00,17007.55,17011.68,17005.52,17009.79,147.49372
2,BTCUSDT,2022-12-06 04:53:00,17009.79,17013.96,17007.55,17010.09,166.51496
3,BTCUSDT,2022-12-06 04:54:00,17010.00,17013.92,17009.43,17011.90,36.54084
4,BTCUSDT,2022-12-06 04:55:00,17011.90,17012.43,17009.36,17010.35,39.08232
...,...,...,...,...,...,...,...
83224,BTCUSDT,2023-02-01 23:55:00,23704.51,23711.98,23693.64,23707.48,123.78742
83225,BTCUSDT,2023-02-01 23:56:00,23708.59,23719.69,23704.29,23714.49,157.83294
83226,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623
83227,BTCUSDT,2023-02-01 23:58:00,23721.50,23736.12,23712.27,23716.12,254.51710


In [8]:
def rsi(close, n=14):
    deltas = np.diff(close)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(close)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(close)):
        delta = deltas[i-1]
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n
        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

In [9]:
def bollinger_bands(close, window_size=20, num_std=2):
    # Rolling mean and standard deviation
    rolling_mean = close.rolling(window_size).mean()
    rolling_std = close.rolling(window_size).std()

    # Upper and lower bands
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)

    return upper_band, rolling_mean, lower_band

In [10]:
def fibonacci_retracement(close, open):
    fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.764, 1.0]
    #    fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    diff = close - open
    levels = np.array([close - level * diff for level in fib_levels])
    df = pd.DataFrame(levels.T, columns=[f"Fibonacci {int(level*100)}%" for level in fib_levels])
    return df

In [11]:
df['RSI'] = rsi(df.Close)
df['upper band'],df['rolling mean'],df['lower band'] = bollinger_bands(df.Close)

df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]


,Name,Time,Open,High,Low,Close,Volume,RSI,upper band,rolling mean,lower band
0,BTCUSDT,2022-12-06 04:51:00,17004.99,17009.01,17002.34,17007.29,93.81524,49.063533,NaN,NaN,NaN
1,BTCUSDT,2022-12-06 04:52:00,17007.55,17011.68,17005.52,17009.79,147.49372,49.063533,NaN,NaN,NaN
2,BTCUSDT,2022-12-06 04:53:00,17009.79,17013.96,17007.55,17010.09,166.51496,49.063533,NaN,NaN,NaN
3,BTCUSDT,2022-12-06 04:54:00,17010.00,17013.92,17009.43,17011.90,36.54084,49.063533,NaN,NaN,NaN
4,BTCUSDT,2022-12-06 04:55:00,17011.90,17012.43,17009.36,17010.35,39.08232,49.063533,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
83224,BTCUSDT,2023-02-01 23:55:00,23704.51,23711.98,23693.64,23707.48,123.78742,46.505557,23724.592115,23706.8360,23689.079885
83225,BTCUSDT,2023-02-01 23:56:00,23708.59,23719.69,23704.29,23714.49,157.83294,50.226418,23719.046968,23705.8820,23692.717032
83226,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623,54.055783,23720.174045,23706.0910,23692.007955
83227,BTCUSDT,2023-02-01 23:58:00,23721.50,23736.12,23712.27,23716.12,254.51710,50.730905,23721.357059,23706.5755,23691.793941


In [7]:
df2 = fibonacci_retracement(df.Close, df.Open)

df2

,Fibonacci 0%,Fibonacci 23%,Fibonacci 38%,Fibonacci 50%,Fibonacci 61%,Fibonacci 76%,Fibonacci 100%
0,17007.29,17006.74720,17006.41140,17006.140,17005.86860,17005.53280,17004.99
1,17009.79,17009.26136,17008.93432,17008.670,17008.40568,17008.07864,17007.55
2,17010.09,17010.01920,17009.97540,17009.940,17009.90460,17009.86080,17009.79
3,17011.90,17011.45160,17011.17420,17010.950,17010.72580,17010.44840,17010.00
4,17010.35,17010.71580,17010.94210,17011.125,17011.30790,17011.53420,17011.90
...,...,...,...,...,...,...,...
83224,23707.48,23706.77908,23706.34546,23705.995,23705.64454,23705.21092,23704.51
83225,23714.49,23713.09760,23712.23620,23711.540,23710.84380,23709.98240,23708.59
83226,23722.29,23720.18252,23718.87874,23717.825,23716.77126,23715.46748,23713.36
83227,23716.12,23717.38968,23718.17516,23718.810,23719.44484,23720.23032,23721.50
